In [1]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

import time

get_ipython().run_line_magic('matplotlib', 'inline')
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
import pickle

#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    data = []
    for res in JSONRes['candles']:
        temp = [((datetime.datetime.fromisoformat(res['time'][:19]))+datetime.timedelta(hours=+9)).strftime('%Y-%m-%d %H:%M')]#indexとなる時間を追加
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data




def get_now_data(time, period_Days, money='USD_JPY', foot = 'D', back_days = 0):
    all_data = []
    
    time = time.replace(hour=7, minute=0, second=0, microsecond=0)
    #入力された日の当日の7時
    
    NY = datetime.timedelta(hours=-14)
    back = datetime.timedelta(days=-back_days)
    
    date_to = time + NY + datetime.timedelta(seconds=0) + back
    date_from = date_to + datetime.timedelta(days=-period_Days)
    
    ret = getCandleDataFromOanda(money, api, date_from, date_to, foot)
    month_data = oandaJsonToPythonList(ret)#取得したデータを格納

    all_data.extend(month_data)#データを追加

    # pandas DataFrameへ変換
    df = pd.DataFrame(all_data)

    df.columns = ['Datetime','Volume', 'Open', 'High', 'Low', 'Close']
    df['Open'] = df['Open'].astype('double')
    df['Close'] = df['Close'].astype('double')
    
    df['Rate']=(df['Close']-df['Open'])/df['Open']
    
    df = df.set_index('Datetime')

    return df
    
def get_csv(back_days=100, days=3650, money='USD_JPY'):
    time = datetime.datetime.now()
    df = get_now_data(time, period_Days=days, money=money, foot = 'D', back_days=back_days)
    date_to = (time+datetime.timedelta(days=-back_days)+datetime.timedelta(days=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(days=-days)+datetime.timedelta(days=-back_days)).strftime("%Y-%m-%d")
    df.to_csv(str(money)+"_"+ date_from + "_" + date_to +'.csv', mode = 'w')
    print('"'+str(money)+"_"+ date_from + "_" + date_to + '.csv', end='", ')
    
    return df

def get_moneys_csv(back_days=100, days=3650):
    moneylist = ['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY']
    for i in range (len(moneylist)):
        get_csv(back_days, days, money=moneylist[i])
        
# def classify(x, c, R=0.001):
#         if x<=-R:
#             c[0]+=1
#             return 0
#         elif -R<x<R:
#             c[1]+=1
#             return 1
#         elif R<=x:
#             c[2]+=1
#             return 2
        
def classify_two(x, c):
        if x<=0:
            c[0]+=1
            return -1
        elif 0<x:
            c[1]+=1
            return 1

def predict(dataname, moneyname, test_days):

    df = pd.read_csv(dataname, index_col='Datetime')
    
    count = [0,0]
    #df['class'] = (df["Rate"]).apply(lambda x: classify(x, count, 0.001))
    df['class'] = (df["Rate"]).apply(lambda x: classify_two(x, count))
    df['class'] = df['class'].shift(-1)
    print("Train Data")
    print(" LOW/HIGH")
    print(count)
    
    X = df.drop('class', axis=1)
    y = df['class']
    X = X.drop(X.index[len(X)-1])
    y = y.dropna()
    
    #X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=False)
    #df_now = pd.read_csv('now_data.csv', index_col='Datetime')
    df_now = get_now_data(datetime.datetime.now(), period_Days=test_days, money=moneyname, foot = 'D', back_days = -1)
    print("money: "+moneyname)
    count = [0,0]
    #df['class'] = (df["Rate"]).apply(lambda x: classify(x, count, 0.001))
    df_now['class'] = (df_now["Rate"]).apply(lambda x: classify_two(x, count))
    df_now['class'] = df_now['class'].shift(-1)
    print("NOW Data from" +str(test_days)+ "days ago")
    print(" LOW/MID/HIGH")
    print(count)
    
    print("learning data")
    print(df.head(1).index.values, end=' to ')
    print(df.tail(1).index.values)
    
    print("")
    print("test data")
    print(df_now.head(1).index.values,end=' to ')
    print(df_now.tail(1).index.values)
    print("")
    X_now = df_now.drop('class', axis=1)
    
    X_next = X_now.tail(1) #最終予測に利用
    y_now = df_now['class']
    X_now = X_now.drop(X_now.index[len(X_now)-1])
    y_now = y_now.dropna()
    
    X_train = X
    y_train = y
    X_test = X_now
    y_test = y_now
    
    
    
    
    names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
             "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
             "Quadratic Discriminant Analysis"]
    classifiers = [
        KNeighborsClassifier(3),
        SVC(kernel="linear", C=0.025),
        SVC(gamma=2, C=1),
        DecisionTreeClassifier(max_depth=10),
        RandomForestClassifier(max_depth=10, n_estimators=10, max_features=1),
        AdaBoostClassifier(),
        GaussianNB(),
        LinearDiscriminantAnalysis(),
        QuadraticDiscriminantAnalysis()]
    
    accuracy_scores = []
    precision_scores = []
    next_class = []
    profit = []
    
    
    for name, clf in zip(names, classifiers):
        print(name)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        result = clf.predict(X_test)
        next_class.extend(clf.predict(X_next))
        
        print('accuracy_score: ', end='')
        print(accuracy_score(y_test, result))
        accuracy_scores.extend([accuracy_score(y_test, result)])
        print('confusion_matrix: ')
        print(confusion_matrix(y_test, result))
        print('precision_score: ', end='')
        print(precision_score(y_test, result, average=None))
        precision_scores.extend([precision_score(y_test, result, average=None)])
        
        possestion = 50000
        pay = 2000
        po_rate = 1.95
        
        win = 0
        lose = 0
        
        for i in range(len(result)):
            possestion -= pay
            if (result[i]==-1):#Low予想のとき
                if (result[i]==y_test[i]): #予測成功
                    possestion += pay*po_rate
                    win+=1
                else:
                    lose+=1

            elif(result[i]==1):#High予想の時
                if (result[i]==y_test[i]): #予測成功
                    possestion += pay*po_rate
                    win+=1
                else:
                    lose+=1
            else:  #Mid予想の時
                possestion += pay
        print(str(len(result))+"days trade")
        print("posestion: "+ str(possestion))
        print("profit: "+str(win*0.95-lose))
        
        print('')
        
        profit.extend([win*0.95-lose])
        
        precision = precision_score(y_test, result, average=None)
        if((win*0.95-lose)>1 and precision[0]!=0 and precision[1]!=0):
            model_name = moneyname + "_" + name + ".sav"
            print("CLEAR: " + model_name)
            pickle.dump(clf, open(model_name, 'wb'))
        
    
    return accuracy_scores, precision_scores, next_class, profit

def make_models(test_days):
    #namelist = ["USD_JPY_2007-02-02_2020-10-10.csv", "EUR_JPY_2007-02-02_2020-10-10.csv", "AUD_JPY_2007-02-02_2020-10-10.csv", "GBP_JPY_2007-02-02_2020-10-10.csv", "NZD_JPY_2007-02-02_2020-10-10.csv", "CAD_JPY_2007-02-02_2020-10-10.csv", "CHF_JPY_2007-02-02_2020-10-10.csv", "ZAR_JPY_2007-02-02_2020-10-10.csv"]
    #moneylist = ['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY', 'Average']
    
    namelist = ["CAD_JPY_2007-02-02_2020-10-10.csv", "CHF_JPY_2007-02-02_2020-10-10.csv", "ZAR_JPY_2007-02-02_2020-10-10.csv"]
    moneylist = ['CAD_JPY', 'CHF_JPY', 'ZAR_JPY', 'Average']
    
    
    df = pd.DataFrame(index=['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY', 'Average'], columns = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
             "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
             "Quadratic Discriminant Analysis"])
    
    df_pre = pd.DataFrame(index=['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY', 'Average'], columns = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
             "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
             "Quadratic Discriminant Analysis"])
    
    df_next = pd.DataFrame(index=['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY'], columns = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
             "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
             "Quadratic Discriminant Analysis"])
    
    df_profit = pd.DataFrame(index=['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY', 'Average'], columns = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
             "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
             "Quadratic Discriminant Analysis"])
    
    for i in range(len(namelist)):
        print(namelist[i])
        acc, pre, next_class, profit = predict(namelist[i], moneylist[i], test_days)
        print(acc)
        df.loc[moneylist[i]]=acc
        df_pre.loc[moneylist[i]]=pre
        df_next.loc[moneylist[i]]=next_class
        df_profit.loc[moneylist[i]]=profit
        
    
    df.loc[moneylist[i+1]] = df.mean()
    df_pre.loc[moneylist[i+1]] = df_pre.mean()
    df_profit.loc[moneylist[i+1]] = df_profit.mean()
    print(df)
    print(df_pre)
    print(df_next)
    print(df_profit)
    df.to_csv('accuracy_result.csv', mode = 'w')
    df_pre.to_csv('precision_result.csv', mode = 'w')
    df_next.to_csv('predict_result.csv', mode = 'w')
    df_profit.to_csv('profit_result.csv', mode = 'w')
  

def calc_profit(predict_result, true_data):
    win=0
    lose=0
    for i in range(len(predict_result)):
            if (predict_result[i]==-1):#Low予想のとき
                if (predict_result[i]==true_data[i]): #予測成功
                    win+=1
                else:
                    lose+=1
            elif(predict_result[i]==1):#High予想の時
                if (predict_result[i]==true_data[i]): #予測成功
                    win+=1
                else:
                    lose+=1
    profit = win*0.95-lose #利率を算出
    values = predict_result*profit #予測結果×利率により予測の価値を算出
    return profit, values

def plot_result(index, predict_result, true_data):
    plt.figure(figsize=(20,5))
    index = index.astype(float)

    color = []
    marker = []
    accuracy = []
    
    for x in range(0,len(predict_result)):
        if true_data.values[x] == predict_result[x]:
            color.extend(['blue'])
            accuracy.extend([1])

        else:
            color.extend(['red'])
            accuracy.extend([0])


    plt.scatter(index.index.values, index["Rate"].values, color=color)

    plt.xticks(rotation=90)
    plt.show()
    
    return accuracy

def ensemble(model_names, moneyname, days):
    
    df_now = get_now_data(datetime.datetime.now(), period_Days=days, money=moneyname, foot = 'D', back_days = 0)
    print("money: "+moneyname)
    count = [0,0]
    
    df_now['class'] = (df_now["Rate"]).apply(lambda x: classify_two(x, count))
    df_now['class'] = df_now['class'].shift(-1)
    print("NOW Data from" +str(days)+ "days ago")
    print(" LOW/MID/HIGH")
    print(count)

    print("")
    print("test data")
    print(df_now.head(1).index.values,end=' to ')
    print(df_now.tail(1).index.values)
    print("")
    X_now = df_now.drop('class', axis=1)
    
    X_next = X_now.tail(1) #最終予測に利用
    y_now = df_now['class']
    X_now = X_now.drop(X_now.index[len(X_now)-1])
    y_now = y_now.dropna()
    
    values = []
    values_sum = []
    win=0
    lose=0
    answer = 0
    
    for x in range(len(model_names)):
        clf = pickle.load(open(model_names[x], 'rb')) #clfをload
        print(model_names[x])
        result = clf.predict(X_now) #predict
         
        #score表示
        print("accuracy_score: ", end="")
        print(accuracy_score(result, y_now))
        print("precision_score: ", end="")
        print(precision_score(result, y_now, average=None))
        print(confusion_matrix(result, y_now))
        
        profit, values = calc_profit(result, y_now) #予測結果について利率と、予測*利率による予測の信頼性を算出
        
        answer += clf.predict(X_next)[0]*profit
        
        if len(values_sum)==0:#1回目は配列をコピー
            values_sum = values
        else:
            values_sum += values #２回目からは加算
        print(values)
        
    
        #plot_result(X_now, result, y_now)
        
    ensemble_result = np.where(values_sum<0, -1, 1) #クラス分類
    
    print("")
    print("ensemble_result: ",end='')
    print(accuracy_score(ensemble_result, y_now.values))
    print(ensemble_result)
    print(y_now.values)
    print(values_sum)
    print("precision_score: ", end="")
    print(precision_score(result, y_now, average=None))
    print("profit: ",end='')
    profit, values = calc_profit(ensemble_result, y_now)
    print(profit)
    print("next: ",end='')
    print(answer)
    print(X_next.head(1))
    accuracy = plot_result(X_now, ensemble_result, y_now)
    
    return accuracy
        
    
def compere_ensemble(models_array, moneys_array, days):
    results = []
    
    for i in range(len(moneys_array)):
        results.append(np.array(ensemble(models_array[i], moneys_array[i], 60)).astype('int'))
    
    np.savetxt('out.csv',results,delimiter=',')
    print(results)
    
def ensemble_set(models_array, moneys_array, days):
    
    for i in range(len(models_array)):
        ensemble(models_array[i], moneys_array[i], days)
    
def main():
    AUD_models = ["AUD_JPY_Decision Tree.sav","AUD_JPY_Linear Discriminant Analysis.sav","AUD_JPY_Naive Bayes.sav","AUD_JPY_Nearest Neighbors.sav", "AUD_JPY_Quadratic Discriminant Analysis.sav"]
    CAD_models = ["CAD_JPY_AdaBoost.sav","CAD_JPY_Decision Tree.sav","CAD_JPY_Quadratic Discriminant Analysis.sav"]
    CHF_models = ["CHF_JPY_Decision Tree.sav","CHF_JPY_Random Forest.sav"]
    EUR_models = ["EUR_JPY_AdaBoost.sav", "EUR_JPY_Naive Bayes.sav"]
    GBP_models = ["GBP_JPY_AdaBoost.sav"]
    #NZD_models = null
    USD_models = ["USD_JPY_AdaBoost.sav", "USD_JPY_Linear Discriminant Analysis.sav"]
    ZAR_models = ["ZAR_JPY_Quadratic Discriminant Analysis.sav"]
    
    #models = [AUD_models, CAD_models, CHF_models, EUR_models, GBP_models, ZAR_models]
    #moneys = ["AUD_JPY", "CAD_JPY", "CHF_JPY", "EUR_JPY", "GBP_JPY", "ZAR_JPY"]
    #########[0.780487805, 0.658536585, 0.756097561, 0.609756098, 0.634146341, 0.658536585]
    
    models = [AUD_models, CAD_models, CHF_models]
    moneys = ["AUD_JPY", "CAD_JPY", "CHF_JPY"]
    
    #ensemble(CHF_models, "CHF_JPY", 60)   #選択した通貨に対してアンサンブル
    #compere_ensemble(models, moneys, 30)   #それぞれの通貨に対してアンサンブルした結果をcsvに返す
    
    ensemble_set(models, moneys, 30)  #指定したmoneyで結果を出す
    
#get_moneys_csv(back_days=31, days=5000) # 指定期間のcsvをsave
make_models(30) #classiferを作成
#ver_2.2 5000days recent 30day model

#main()

USD_JPY_2007-02-02_2020-10-10.csv
Train Data
 LOW/HIGH
[2138, 1932]
money: USD_JPY
NOW Data from30days ago
 LOW/MID/HIGH
[12, 10]
learning data
['2007-02-02 07:00'] to ['2020-10-09 06:00']

test data
['2020-10-12 06:00'] to ['2020-11-10 07:00']

Nearest Neighbors
accuracy_score: 0.2857142857142857
confusion_matrix: 
[[3 8]
 [7 3]]
precision_score: [0.3        0.27272727]
21days trade
posestion: 31400.0
profit: -9.3

Linear SVM
accuracy_score: 0.5238095238095238
confusion_matrix: 
[[11  0]
 [10  0]]
precision_score: [0.52380952 0.        ]
21days trade
posestion: 50900.0
profit: 0.4499999999999993

RBF SVM


C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy_score: 0.5238095238095238
confusion_matrix: 
[[11  0]
 [10  0]]
precision_score: [0.52380952 0.        ]
21days trade
posestion: 50900.0
profit: 0.4499999999999993

Decision Tree
accuracy_score: 0.3333333333333333
confusion_matrix: 
[[2 9]
 [5 5]]
precision_score: [0.28571429 0.35714286]
21days trade
posestion: 35300.0
profit: -7.3500000000000005

Random Forest
accuracy_score: 0.5238095238095238
confusion_matrix: 
[[4 7]
 [3 7]]
precision_score: [0.57142857 0.5       ]
21days trade
posestion: 50900.0
profit: 0.4499999999999993

AdaBoost


C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy_score: 0.5238095238095238
confusion_matrix: 
[[4 7]
 [3 7]]
precision_score: [0.57142857 0.5       ]
21days trade
posestion: 50900.0
profit: 0.4499999999999993

Naive Bayes
accuracy_score: 0.47619047619047616
confusion_matrix: 
[[ 0 11]
 [ 0 10]]
precision_score: [0.         0.47619048]
21days trade
posestion: 47000.0
profit: -1.5

Linear Discriminant Analysis
accuracy_score: 0.5238095238095238
confusion_matrix: 
[[7 4]
 [6 4]]
precision_score: [0.53846154 0.5       ]
21days trade
posestion: 50900.0
profit: 0.4499999999999993

Quadratic Discriminant Analysis
accuracy_score: 0.5238095238095238
confusion_matrix: 
[[9 2]
 [8 2]]
precision_score: [0.52941176 0.5       ]
21days trade
posestion: 50900.0
profit: 0.4499999999999993

[0.2857142857142857, 0.5238095238095238, 0.5238095238095238, 0.3333333333333333, 0.5238095238095238, 0.5238095238095238, 0.47619047619047616, 0.5238095238095238, 0.5238095238095238]
EUR_JPY_2007-02-02_2020-10-10.csv
Train Data
 LOW/HIGH
[2090, 1979]


...

C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


money: NZD_JPY
NOW Data from30days ago
 LOW/MID/HIGH
[8, 14]
learning data
['2007-02-02 07:00'] to ['2020-10-09 06:00']

test data
['2020-10-12 06:00'] to ['2020-11-10 07:00']

Nearest Neighbors
accuracy_score: 0.6666666666666666
confusion_matrix: 
[[ 3  4]
 [ 3 11]]
precision_score: [0.5        0.73333333]
21days trade
posestion: 62600.0
profit: 6.299999999999999

CLEAR: NZD_JPY_Nearest Neighbors.sav
Linear SVM
accuracy_score: 0.47619047619047616
confusion_matrix: 
[[1 6]
 [5 9]]
precision_score: [0.16666667 0.6       ]
21days trade
posestion: 47000.0
profit: -1.5

RBF SVM
accuracy_score: 0.6666666666666666
confusion_matrix: 
[[ 0  7]
 [ 0 14]]
precision_score: [0.         0.66666667]
21days trade
posestion: 62600.0
profit: 6.299999999999999

Decision Tree
accuracy_score: 0.6666666666666666
confusion_matrix: 
[[ 0  7]
 [ 0 14]]
precision_score: [0.         0.66666667]
21days trade
posestion: 62600.0
profit: 6.299999999999999

Random Forest
accuracy_score: 0.5238095238095238
confusion_

C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy_score: 0.38095238095238093
confusion_matrix: 
[[2 5]
 [8 6]]
precision_score: [0.2        0.54545455]
21days trade
posestion: 39200.0
profit: -5.4

Naive Bayes
accuracy_score: 0.47619047619047616
confusion_matrix: 
[[5 2]
 [9 5]]
precision_score: [0.35714286 0.71428571]
21days trade
posestion: 47000.0
profit: -1.5

Linear Discriminant Analysis
accuracy_score: 0.5714285714285714
confusion_matrix: 
[[ 0  7]
 [ 2 12]]
precision_score: [0.         0.63157895]
21days trade
posestion: 54800.0
profit: 2.3999999999999986

Quadratic Discriminant Analysis
accuracy_score: 0.5714285714285714
confusion_matrix: 
[[5 2]
 [7 7]]
precision_score: [0.41666667 0.77777778]
21days trade
posestion: 54800.0
profit: 2.3999999999999986

CLEAR: NZD_JPY_Quadratic Discriminant Analysis.sav
[0.6666666666666666, 0.47619047619047616, 0.6666666666666666, 0.6666666666666666, 0.5238095238095238, 0.38095238095238093, 0.47619047619047616, 0.5714285714285714, 0.5714285714285714]
CAD_JPY_2007-02-02_2020-10-10.csv


V20Error: {"errorMessage":"Invalid value specified for 'to'. Time is in the future"}

In [2]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

import time

get_ipython().run_line_magic('matplotlib', 'inline')
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
import pickle

#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    data = []
    for res in JSONRes['candles']:
        temp = [((datetime.datetime.fromisoformat(res['time'][:19]))+datetime.timedelta(hours=+9)).strftime('%Y-%m-%d %H:%M')]#indexとなる時間を追加
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data




def get_now_data(time, period_Days, money='USD_JPY', foot = 'D', back_days = 0):
    all_data = []
    
    time = time.replace(hour=7, minute=0, second=0, microsecond=0)
    #入力された日の当日の7時
    
    NY = datetime.timedelta(hours=-14)
    back = datetime.timedelta(days=-back_days)
    
    date_to = time + NY + datetime.timedelta(seconds=0) + back
    date_from = date_to + datetime.timedelta(days=-period_Days)
    
    ret = getCandleDataFromOanda(money, api, date_from, date_to, foot)
    month_data = oandaJsonToPythonList(ret)#取得したデータを格納

    all_data.extend(month_data)#データを追加

    # pandas DataFrameへ変換
    df = pd.DataFrame(all_data)

    df.columns = ['Datetime','Volume', 'Open', 'High', 'Low', 'Close']
    df['Open'] = df['Open'].astype('double')
    df['Close'] = df['Close'].astype('double')
    
    df['Rate']=(df['Close']-df['Open'])/df['Open']
    
    df = df.set_index('Datetime')

    return df
    
def get_csv(back_days=100, days=3650, money='USD_JPY'):
    time = datetime.datetime.now()
    df = get_now_data(time, period_Days=days, money=money, foot = 'D', back_days=back_days)
    date_to = (time+datetime.timedelta(days=-back_days)+datetime.timedelta(days=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(days=-days)+datetime.timedelta(days=-back_days)).strftime("%Y-%m-%d")
    df.to_csv(str(money)+"_"+ date_from + "_" + date_to +'.csv', mode = 'w')
    print('"'+str(money)+"_"+ date_from + "_" + date_to + '.csv', end='", ')
    
    return df

def get_moneys_csv(back_days=100, days=3650):
    moneylist = ['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY']
    for i in range (len(moneylist)):
        get_csv(back_days, days, money=moneylist[i])
        
# def classify(x, c, R=0.001):
#         if x<=-R:
#             c[0]+=1
#             return 0
#         elif -R<x<R:
#             c[1]+=1
#             return 1
#         elif R<=x:
#             c[2]+=1
#             return 2
        
def classify_two(x, c):
        if x<=0:
            c[0]+=1
            return -1
        elif 0<x:
            c[1]+=1
            return 1

def predict(dataname, moneyname, test_days):

    df = pd.read_csv(dataname, index_col='Datetime')
    
    count = [0,0]
    #df['class'] = (df["Rate"]).apply(lambda x: classify(x, count, 0.001))
    df['class'] = (df["Rate"]).apply(lambda x: classify_two(x, count))
    df['class'] = df['class'].shift(-1)
    print("Train Data")
    print(" LOW/HIGH")
    print(count)
    
    X = df.drop('class', axis=1)
    y = df['class']
    X = X.drop(X.index[len(X)-1])
    y = y.dropna()
    
    #X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=False)
    #df_now = pd.read_csv('now_data.csv', index_col='Datetime')
    df_now = get_now_data(datetime.datetime.now(), period_Days=test_days, money=moneyname, foot = 'D', back_days = 1)
    print("money: "+moneyname)
    count = [0,0]
    #df['class'] = (df["Rate"]).apply(lambda x: classify(x, count, 0.001))
    df_now['class'] = (df_now["Rate"]).apply(lambda x: classify_two(x, count))
    df_now['class'] = df_now['class'].shift(-1)
    print("NOW Data from" +str(test_days)+ "days ago")
    print(" LOW/MID/HIGH")
    print(count)
    
    print("learning data")
    print(df.head(1).index.values, end=' to ')
    print(df.tail(1).index.values)
    
    print("")
    print("test data")
    print(df_now.head(1).index.values,end=' to ')
    print(df_now.tail(1).index.values)
    print("")
    X_now = df_now.drop('class', axis=1)
    
    X_next = X_now.tail(1) #最終予測に利用
    y_now = df_now['class']
    X_now = X_now.drop(X_now.index[len(X_now)-1])
    y_now = y_now.dropna()
    
    X_train = X
    y_train = y
    X_test = X_now
    y_test = y_now
    
    
    
    
    names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
             "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
             "Quadratic Discriminant Analysis"]
    classifiers = [
        KNeighborsClassifier(3),
        SVC(kernel="linear", C=0.025),
        SVC(gamma=2, C=1),
        DecisionTreeClassifier(max_depth=10),
        RandomForestClassifier(max_depth=10, n_estimators=10, max_features=1),
        AdaBoostClassifier(),
        GaussianNB(),
        LinearDiscriminantAnalysis(),
        QuadraticDiscriminantAnalysis()]
    
    accuracy_scores = []
    precision_scores = []
    next_class = []
    profit = []
    
    
    for name, clf in zip(names, classifiers):
        print(name)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        result = clf.predict(X_test)
        next_class.extend(clf.predict(X_next))
        
        print('accuracy_score: ', end='')
        print(accuracy_score(y_test, result))
        accuracy_scores.extend([accuracy_score(y_test, result)])
        print('confusion_matrix: ')
        print(confusion_matrix(y_test, result))
        print('precision_score: ', end='')
        print(precision_score(y_test, result, average=None))
        precision_scores.extend([precision_score(y_test, result, average=None)])
        
        possestion = 50000
        pay = 2000
        po_rate = 1.95
        
        win = 0
        lose = 0
        
        for i in range(len(result)):
            possestion -= pay
            if (result[i]==-1):#Low予想のとき
                if (result[i]==y_test[i]): #予測成功
                    possestion += pay*po_rate
                    win+=1
                else:
                    lose+=1

            elif(result[i]==1):#High予想の時
                if (result[i]==y_test[i]): #予測成功
                    possestion += pay*po_rate
                    win+=1
                else:
                    lose+=1
            else:  #Mid予想の時
                possestion += pay
        print(str(len(result))+"days trade")
        print("posestion: "+ str(possestion))
        print("profit: "+str(win*0.95-lose))
        
        print('')
        
        profit.extend([win*0.95-lose])
        
        precision = precision_score(y_test, result, average=None)
        if((win*0.95-lose)>1 and precision[0]!=0 and precision[1]!=0):
            model_name = moneyname + "_" + name + ".sav"
            print("CLEAR: " + model_name)
            pickle.dump(clf, open(model_name, 'wb'))
        
    
    return accuracy_scores, precision_scores, next_class, profit

def make_models(test_days):
    #namelist = ["USD_JPY_2007-02-02_2020-10-10.csv", "EUR_JPY_2007-02-02_2020-10-10.csv", "AUD_JPY_2007-02-02_2020-10-10.csv", "GBP_JPY_2007-02-02_2020-10-10.csv", "NZD_JPY_2007-02-02_2020-10-10.csv", "CAD_JPY_2007-02-02_2020-10-10.csv", "CHF_JPY_2007-02-02_2020-10-10.csv", "ZAR_JPY_2007-02-02_2020-10-10.csv"]
    #moneylist = ['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY', 'Average']
    
    #namelist = ["CAD_JPY_2007-02-02_2020-10-10.csv", "CHF_JPY_2007-02-02_2020-10-10.csv", "ZAR_JPY_2007-02-02_2020-10-10.csv"]
    #moneylist = ['CAD_JPY', 'CHF_JPY', 'ZAR_JPY', 'Average']
    
    namelist = ["ZAR_JPY_2007-02-02_2020-10-10.csv"]
    moneylist = ["ZAR_JPY"]
    
    df = pd.DataFrame(index=['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY', 'Average'], columns = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
             "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
             "Quadratic Discriminant Analysis"])
    
    df_pre = pd.DataFrame(index=['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY', 'Average'], columns = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
             "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
             "Quadratic Discriminant Analysis"])
    
    df_next = pd.DataFrame(index=['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY'], columns = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
             "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
             "Quadratic Discriminant Analysis"])
    
    df_profit = pd.DataFrame(index=['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY', 'Average'], columns = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
             "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
             "Quadratic Discriminant Analysis"])
    
    for i in range(len(namelist)):
        print(namelist[i])
        acc, pre, next_class, profit = predict(namelist[i], moneylist[i], test_days)
        print(acc)
        df.loc[moneylist[i]]=acc
        df_pre.loc[moneylist[i]]=pre
        df_next.loc[moneylist[i]]=next_class
        df_profit.loc[moneylist[i]]=profit
        
    
    df.loc[moneylist[i+1]] = df.mean()
    df_pre.loc[moneylist[i+1]] = df_pre.mean()
    df_profit.loc[moneylist[i+1]] = df_profit.mean()
    print(df)
    print(df_pre)
    print(df_next)
    print(df_profit)
    df.to_csv('accuracy_result.csv', mode = 'w')
    df_pre.to_csv('precision_result.csv', mode = 'w')
    df_next.to_csv('predict_result.csv', mode = 'w')
    df_profit.to_csv('profit_result.csv', mode = 'w')
  

def calc_profit(predict_result, true_data):
    win=0
    lose=0
    for i in range(len(predict_result)):
            if (predict_result[i]==-1):#Low予想のとき
                if (predict_result[i]==true_data[i]): #予測成功
                    win+=1
                else:
                    lose+=1
            elif(predict_result[i]==1):#High予想の時
                if (predict_result[i]==true_data[i]): #予測成功
                    win+=1
                else:
                    lose+=1
    profit = win*0.95-lose #利率を算出
    values = predict_result*profit #予測結果×利率により予測の価値を算出
    return profit, values

def plot_result(index, predict_result, true_data):
    plt.figure(figsize=(20,5))
    index = index.astype(float)

    color = []
    marker = []
    accuracy = []
    
    for x in range(0,len(predict_result)):
        if true_data.values[x] == predict_result[x]:
            color.extend(['blue'])
            accuracy.extend([1])

        else:
            color.extend(['red'])
            accuracy.extend([0])


    plt.scatter(index.index.values, index["Rate"].values, color=color)

    plt.xticks(rotation=90)
    plt.show()
    
    return accuracy

def ensemble(model_names, moneyname, days):
    
    df_now = get_now_data(datetime.datetime.now(), period_Days=days, money=moneyname, foot = 'D', back_days = 0)
    print("money: "+moneyname)
    count = [0,0]
    
    df_now['class'] = (df_now["Rate"]).apply(lambda x: classify_two(x, count))
    df_now['class'] = df_now['class'].shift(-1)
    print("NOW Data from" +str(days)+ "days ago")
    print(" LOW/MID/HIGH")
    print(count)

    print("")
    print("test data")
    print(df_now.head(1).index.values,end=' to ')
    print(df_now.tail(1).index.values)
    print("")
    X_now = df_now.drop('class', axis=1)
    
    X_next = X_now.tail(1) #最終予測に利用
    y_now = df_now['class']
    X_now = X_now.drop(X_now.index[len(X_now)-1])
    y_now = y_now.dropna()
    
    values = []
    values_sum = []
    win=0
    lose=0
    answer = 0
    
    for x in range(len(model_names)):
        clf = pickle.load(open(model_names[x], 'rb')) #clfをload
        print(model_names[x])
        result = clf.predict(X_now) #predict
         
        #score表示
        print("accuracy_score: ", end="")
        print(accuracy_score(result, y_now))
        print("precision_score: ", end="")
        print(precision_score(result, y_now, average=None))
        print(confusion_matrix(result, y_now))
        
        profit, values = calc_profit(result, y_now) #予測結果について利率と、予測*利率による予測の信頼性を算出
        
        answer += clf.predict(X_next)[0]*profit
        
        if len(values_sum)==0:#1回目は配列をコピー
            values_sum = values
        else:
            values_sum += values #２回目からは加算
        print(values)
        
    
        #plot_result(X_now, result, y_now)
        
    ensemble_result = np.where(values_sum<0, -1, 1) #クラス分類
    
    print("")
    print("ensemble_result: ",end='')
    print(accuracy_score(ensemble_result, y_now.values))
    print(ensemble_result)
    print(y_now.values)
    print(values_sum)
    print("precision_score: ", end="")
    print(precision_score(result, y_now, average=None))
    print("profit: ",end='')
    profit, values = calc_profit(ensemble_result, y_now)
    print(profit)
    print("next: ",end='')
    print(answer)
    print(X_next.head(1))
    accuracy = plot_result(X_now, ensemble_result, y_now)
    
    return accuracy
        
    
def compere_ensemble(models_array, moneys_array, days):
    results = []
    
    for i in range(len(moneys_array)):
        results.append(np.array(ensemble(models_array[i], moneys_array[i], 60)).astype('int'))
    
    np.savetxt('out.csv',results,delimiter=',')
    print(results)
    
def ensemble_set(models_array, moneys_array, days):
    
    for i in range(len(models_array)):
        ensemble(models_array[i], moneys_array[i], days)
    
def main():
    AUD_models = ["AUD_JPY_Decision Tree.sav","AUD_JPY_Linear Discriminant Analysis.sav","AUD_JPY_Naive Bayes.sav","AUD_JPY_Nearest Neighbors.sav", "AUD_JPY_Quadratic Discriminant Analysis.sav"]
    CAD_models = ["CAD_JPY_AdaBoost.sav","CAD_JPY_Decision Tree.sav","CAD_JPY_Quadratic Discriminant Analysis.sav"]
    CHF_models = ["CHF_JPY_Decision Tree.sav","CHF_JPY_Random Forest.sav"]
    EUR_models = ["EUR_JPY_AdaBoost.sav", "EUR_JPY_Naive Bayes.sav"]
    GBP_models = ["GBP_JPY_AdaBoost.sav"]
    #NZD_models = null
    USD_models = ["USD_JPY_AdaBoost.sav", "USD_JPY_Linear Discriminant Analysis.sav"]
    ZAR_models = ["ZAR_JPY_Quadratic Discriminant Analysis.sav"]
    
    #models = [AUD_models, CAD_models, CHF_models, EUR_models, GBP_models, ZAR_models]
    #moneys = ["AUD_JPY", "CAD_JPY", "CHF_JPY", "EUR_JPY", "GBP_JPY", "ZAR_JPY"]
    #########[0.780487805, 0.658536585, 0.756097561, 0.609756098, 0.634146341, 0.658536585]
    
    models = [AUD_models, CAD_models, CHF_models]
    moneys = ["AUD_JPY", "CAD_JPY", "CHF_JPY"]
    
    #ensemble(CHF_models, "CHF_JPY", 60)   #選択した通貨に対してアンサンブル
    #compere_ensemble(models, moneys, 30)   #それぞれの通貨に対してアンサンブルした結果をcsvに返す
    
    ensemble_set(models, moneys, 30)  #指定したmoneyで結果を出す
    
#get_moneys_csv(back_days=31, days=5000) # 指定期間のcsvをsave
make_models(30) #classiferを作成
#ver_2.2 5000days recent 30day model

#main()

ZAR_JPY_2007-02-02_2020-10-10.csv
Train Data
 LOW/HIGH
[2077, 1976]
money: ZAR_JPY
NOW Data from30days ago
 LOW/MID/HIGH
[6, 16]
learning data
['2007-02-02 07:00'] to ['2020-10-09 06:00']

test data
['2020-10-13 06:00'] to ['2020-11-11 07:00']

Nearest Neighbors
accuracy_score: 0.38095238095238093
confusion_matrix: 
[[2 4]
 [9 6]]
precision_score: [0.18181818 0.6       ]
21days trade
posestion: 39200.0
profit: -5.4

Linear SVM
accuracy_score: 0.7142857142857143
confusion_matrix: 
[[ 0  6]
 [ 0 15]]
precision_score: [0.         0.71428571]
21days trade
posestion: 66500.0
profit: 8.25

RBF SVM


C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy_score: 0.23809523809523808
confusion_matrix: 
[[ 4  2]
 [14  1]]
precision_score: [0.22222222 0.33333333]
21days trade
posestion: 27500.0
profit: -11.25

Decision Tree
accuracy_score: 0.38095238095238093
confusion_matrix: 
[[ 3  3]
 [10  5]]
precision_score: [0.23076923 0.625     ]
21days trade
posestion: 39200.0
profit: -5.4

Random Forest
accuracy_score: 0.5714285714285714
confusion_matrix: 
[[4 2]
 [7 8]]
precision_score: [0.36363636 0.8       ]
21days trade
posestion: 54800.0
profit: 2.3999999999999986

CLEAR: ZAR_JPY_Random Forest.sav
AdaBoost
accuracy_score: 0.7142857142857143
confusion_matrix: 
[[ 0  6]
 [ 0 15]]
precision_score: [0.         0.71428571]
21days trade
posestion: 66500.0
profit: 8.25

Naive Bayes
accuracy_score: 0.7142857142857143
confusion_matrix: 
[[ 0  6]
 [ 0 15]]
precision_score: [0.         0.71428571]
21days trade
posestion: 66500.0
profit: 8.25

Linear Discriminant Analysis
accuracy_score: 0.7142857142857143
confusion_matrix: 
[[ 0  6]
 [ 0 15]]
pr

C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


IndexError: list index out of range